In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "./dataset"

dataset = datasets.ImageFolder(root=dataset_path, transform=image_transforms)
len(dataset)

2125

In [5]:
class_names = dataset.classes
class_names 

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [6]:
num_classes = len(dataset.classes)
num_classes

6

In [7]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

train_size, val_size

(1593, 532)

In [8]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

### Model Training & Hyperparameter Tunning

In [10]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [11]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    # Load the model
    model = CarClassifierResNet(num_classes=num_classes, dropout_rate=dropout_rate).to(device)
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    # Training loop (using fewer epochs for faster hyperparameter tuning)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Report intermediate result to Optuna
        trial.report(accuracy, epoch)
        
        # Handle pruning (if applicable)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end - start} seconds")
    
    return accuracy

In [12]:
# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-12-04 22:49:49,645] A new study created in memory with name: no-name-a5bcb6fa-a27b-4c6c-acd5-19d360fe65b4
[I 2025-12-04 23:02:13,736] Trial 0 finished with value: 46.42857142857143 and parameters: {'lr': 1.2842461759373165e-05, 'dropout_rate': 0.5103907439213591}. Best is trial 0 with value: 46.42857142857143.


Execution time: 742.6606478691101 seconds


[I 2025-12-04 23:15:51,198] Trial 1 finished with value: 50.93984962406015 and parameters: {'lr': 1.8265172480601113e-05, 'dropout_rate': 0.6044915344663946}. Best is trial 1 with value: 50.93984962406015.


Execution time: 815.3702430725098 seconds


[I 2025-12-04 23:29:12,271] Trial 2 finished with value: 75.56390977443608 and parameters: {'lr': 0.0004911071869783563, 'dropout_rate': 0.36868648795263226}. Best is trial 2 with value: 75.56390977443608.


...

[I 2025-12-05 00:22:11,112] Trial 10 pruned. 
[I 2025-12-05 00:30:07,771] Trial 11 finished with value: 75.56390977443608 and parameters: {'lr': 0.0009718953359387436, 'dropout_rate': 0.23732755336291725}. Best is trial 9 with value: 78.19548872180451.


Execution time: 475.85426902770996 seconds


[I 2025-12-05 00:32:37,730] Trial 12 pruned. 
[I 2025-12-05 00:36:56,910] Trial 13 pruned. 
[I 2025-12-05 00:38:53,614] Trial 14 pruned. 
[I 2025-12-05 00:44:45,617] Trial 15 finished with value: 75.56390977443608 and parameters: {'lr': 0.0005890405555838522, 'dropout_rate': 0.29296471266962587}. Best is trial 9 with value: 78.19548872180451.


Execution time: 351.30571460723877 seconds


[I 2025-12-05 00:46:46,073] Trial 16 pruned. 
[I 2025-12-05 00:48:46,137] Trial 17 pruned. 
[I 2025-12-05 00:50:56,831] Trial 18 pruned. 
[I 2025-12-05 00:52:59,928] Trial 19 pruned. 


In [13]:
study.best_params

{'lr': 0.00028981552779041165, 'dropout_rate': 0.619671660133448}

In the first run it gave me best results with lr=0.005 and dropout=0.2. In the second run it gave different results as you see it above. Both the runs are not making much difference in terms of accuracy hence I will just use the results from the first run